In [1]:
import CSV
using DataFrames
# using StatsBase
include("pred_funcs.jl")
using MLJ
# import MLJBase
import TableView.showtable
using Plots
import FileIO
import Statistics
using LazyArrays
using Zygote
import LinearAlgebra

WebIO._IJuliaInit()

In [2]:
using Turing, Distributions

using MCMCChains, StatsPlots

using StatsFuns: logistic

using Random
Random.seed!(0);

using StatsBase
import MLJBase

In [401]:
MissingImputator = @load MissingImputator pkg="BetaML" verbosity=0
XGBoostClassifier = @load XGBoostClassifier verbosity=0  # loads code defining a model type
LogisticClassifier = @load LogisticClassifier pkg="MLJLinearModels" verbosity=0;  # loads code defining a model type
AdaBoostStumpClassifier = @load AdaBoostStumpClassifier verbosity=0;  # loads code defining a model type
EvoTreeClassifier = @load EvoTreeClassifier verbosity=0
GradientBoostingClassifier = @load GradientBoostingClassifier verbosity=0
EvoTreeGaussian = @load EvoTreeGaussian verbosity=0
SVC = @load SVC verbosity=0

EvoTreeRegressor = @load EvoTreeRegressor verbosity=0
EvoTreeCount = @load EvoTreeCount verbosity=0

NeuralNetworkClassifier = @load NeuralNetworkClassifier


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/alexpanchot/.julia/packages/MLJModels/4sRmw/src/loading.jl:168


import MLJFlux ✔


MLJFlux.NeuralNetworkClassifier

In [402]:
# run(`unzip data_full.zip`)
# run(`rm data_full.jld2`)

In [403]:

sample_key2, xold, yold = FileIO.load("data_full.jld2","sample_key","x" ,"y");

xold.col1=collect(1:nrow(xold));
println(nrow(xold))
# filter!(x->x.ALT_I>0.5 || x.REF_I>0.5,xold) # indel
# filter!(x->x.ALT_I<0.5 && x.REF_I<0.5,xold) # SNV
println(nrow(xold))
filter!(x->x.training>0.5,xold)
# select!(xold,Not("training"));
println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Mutect),xold);
println(nrow(xold))

1523404
1523404
1453749
3255


In [404]:
# ["gt_AF_Mutect","MPOS_Mutect","TLOD_Mutect", "alt_len","MBQ_Mutect_2","pon_pvalue_Mutect" ]

# density(collect(skipmissing(filter(x->x.training > 0.5,xold)[:,"pon_pvalue_Mutect"])),alpha=1.0)
# density!(collect(skipmissing(filter(x->x.training < 0.5,xold)[:,"pon_pvalue_Mutect"])),alpha=1.0)

In [405]:
# showtable(sample_key2[findall(x->x,[i==true for i in yold]),:]);

In [406]:
for j in 1:ncol(xold), i in 1:nrow(xold)
    if !ismissing(xold[i,j]) && (isnan(xold[i,j]) || isinf(xold[i,j]) )
        xold[i,j] = missing
    end
end

In [407]:
coldrop = ["FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(xold)
    if i == "col1" || i=="training"
        continue
    end
    if length(unique(xold[:,i])) < 2 || occursin("Pindel",i) || occursin("Lofreq",i) || occursin("Vardict",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop);

In [408]:
select!(xold, Not(coldrop));
println(sum([i==true for i in yold]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))

println(size(yold))
println(length(sample_key2))
println()
yold = yold[xold.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
sample_key2 = sample_key2[xold.col1]
select!(xold,Not("col1"));

println(size(xold))
println(size(yold))
println(length(sample_key2))
println(sum([i==true for i in yold]))

226
(1523404,)
1523404

(3255, 64)
(3255,)
3255
67


In [409]:
# istrain=0
# for i in 1:nrow(xold)
#     if sum(ismissing.(Vector(xold[i,:]))) > 0
# #         if xold[i,:training] > 0.5
# #             istrain += 1
# #         end
#         println(sample_key[i]," ",i)
#     end
# end
# println(istrain)

In [410]:
# showtable(xold[60:65,:])

In [411]:
# for col in names(xold)
#     println(col,"  ",1-sum(ismissing.(xold[:,col]))/nrow(xold) )
# end

In [412]:
# xold.sample_key=sample_key
# xold.real=[i==true for i in yold]
# filter(x->x.training>0.5 && x.real && (ismissing(x.MPOS_Mutect) || x.MPOS_Mutect < 30),xold)[:,["MPOS_Mutect","sample_key","real"]]

In [413]:
# namd=names(xsnv)[13] # 2,6,8,9,13
# namd="MBQ_Mutect_1"
# namd="MPOS_Mutect"
# xold.sample_key=sample_key
# xold.real=[i==true for i in yold]
# print(namd)
# histogram(collect(skipmissing(filter(x->x.training>0.5 && x.real,xold)[:,namd])),legend=true,label="TT",alpha=0.3,bins=10)
# histogram!(collect(skipmissing(filter(x->x.training<0.5 && x.real,xold)[:,namd])),label="BMT",alpha=0.3,bins=10)



In [414]:
# ysnv[[9, 12, 13, 14, 21, 24, 25, 26, 40, 41, 42, 44]];

In [415]:
# ysnv[[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 15,16]];

In [416]:
xsnv = copy(xold);
ysnv = copy(yold);

# train2 = findall(x->x,[i>0.5 for i in xsnv.training])
# test2 = findall(x->x,[i<0.5 for i in xsnv.training])

# train2 = [9, 12, 13, 14, 21, 24, 25, 26, 40, 41, 42, 44,
# 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 15,16
# ]

# test2=setdiff(collect(1:length(ysnv)),train2)

train2, test2 = partition(eachindex(ysnv), 0.5, stratify=ysnv,  shuffle=true, rng=12344);

select!(xsnv,Not("training"));

In [417]:
transform_standardizer!(xsnv,fit_standardizer(xsnv[train2,:]));

In [418]:
# showtable(xsnv[1:10,:])

In [419]:
sum(ismissing.(Matrix(xsnv)))

7288

In [420]:
stmach = machine(MissingImputator(K=5),xsnv[train2,:]) 
@time sttran = fit!(stmach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


  2.328190 seconds (2.18 M allocations: 342.594 MiB, 61.74% gc time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @134 ⏎ `Table{Union{AbstractVector{Union{Missing, ScientificTypesBase.Continuous}}, AbstractVector{ScientificTypesBase.Continuous}}}`


In [421]:
tt = MLJ.transform(sttran,xsnv);
for i in 1:ncol(xsnv)
    xsnv[:,i] = tt[i]
end

In [422]:
disallowmissing!(xsnv);

In [423]:
sample_key = copy(sample_key2)
train = copy(train2)
test = copy(test2)
x = copy(xsnv);
y = copy(ysnv);

In [424]:
println(size(x[train,:])," ",sum([i==true for i in y[train]] ) )
println(size(x[test,:])," ",sum([i==true for i in y[test]] ) )
println(size(y[train]))
println(size(sample_key))

(1628, 63) 34
(1627, 63) 33
(1628,)
(3255,)


In [425]:
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

ratio: 0.02132998745294856
number of new: 285


In [426]:
y = coerce(copy(y),OrderedFactor);

In [427]:
println(size(x[train,:])," ",sum([i==true for i in y[train]] ) )
println(size(x[test,:])," ",sum([i==true for i in y[test]] ) )
println(size(y[train]))
println(size(sample_key))

(1913, 63) 319
(1627, 63) 33
(1913,)
(3540,)


# MLJ Models

In [363]:
# inputvars = ["TLOD_Mutect",
# #     "MPOS_Mutect","MBQ_Mutect_2",
#     "dust_score_Mutect",
#     "dust_score_3_Mutect",
#     "dust_score_5_Mutect","dust_score_10_Mutect","alt_len","VARIANT_CLASS_SNV"
# ]


In [430]:
info(NeuralNetworkClassifier)

A neural network model for making probabilistic predictions of a `Multiclass` or `OrderedFactor` target, given a table of `Continuous` features. 
→ based on [MLJFlux](https://github.com/alan-turing-institute/MLJFlux.jl).
→ do `@load NeuralNetworkClassifier pkg="MLJFlux"` to use the model.
→ do `?NeuralNetworkClassifier` for documentation.
(name = "NeuralNetworkClassifier",
 package_name = "MLJFlux",
 is_supervised = true,
 abstract_type = Probabilistic,
 deep_properties = (:optimiser, :builder),
 docstring = "A neural network model for making probabilistic predictions of a `Multiclass` or `OrderedFactor` target, given a table of `Continuous` features. \n→ based on [MLJFlux](https://github.com/alan-turing-institute/MLJFlux.jl).\n→ do `@load NeuralNetworkClassifier pkg=\"MLJFlux\"` to use the model.\n→ do `?NeuralNetworkClassifier` for documentation.",
 fit_data_scitype = Tuple{Table{var"#s53"} where var"#s53"<:(AbstractVector{var"#s52"} where var"#s52"<:ScientificTypesBase.Continuous), 

In [436]:
NeuralNetworkClassifier(builder = (
			n_hidden = 1,
			dropout = 0.5,))

NeuralNetworkClassifier(
    builder = (n_hidden = 1, dropout = 0.5),
    finaliser = NNlib.softmax,
    optimiser = Flux.Optimise.ADAM(0.001, (0.9, 0.999), IdDict{Any, Any}()),
    loss = Flux.Losses.crossentropy,
    epochs = 10,
    batch_size = 1,
    lambda = 0.0,
    alpha = 0.0,
    rng = Random._GLOBAL_RNG(),
    optimiser_changes_trigger_retraining = false,
    acceleration = CPU1{Nothing}(nothing))

In [437]:

r = [
#     range(EvoTreeClassifier(), :max_depth, lower=1,upper=4),
#     range(EvoTreeClassifier(), :η, lower=0.0,upper=1.0),
#     range(EvoTreeClassifier(), :γ, lower=0.0,upper=1.0),
#     range(EvoTreeClassifier(), :λ, lower=0.0,upper=1.0),
#     range(EvoTreeClassifier(), :α, lower=0.0,upper=1.0),
#     range(EvoTreeClassifier(), :min_weight, lower=0.0,upper=1.0),
#     range(EvoTreeClassifier(), :nrounds, lower=1,upper=10),
#     range(EvoTreeClassifier(), :nbins, lower=4, upper=128),
    
    
#     range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
#     range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
    
#     range(SVC(), :degree, lower=1,upper=4),
#     range(SVC(), :gamma, lower=0.0,upper=1.0),   
    
    range(NeuralNetworkClassifier(), :alpha, lower=0.0, upper=1.0),
]

1-element Vector{MLJBase.NumericRange{Float64, MLJBase.Bounded, Symbol}}:
 NumericRange(0.0 ≤ alpha ≤ 1.0; origin=0.5, unit=0.5)

In [438]:
:linear, :logistic, :poisson, :L1, :quantile, :softmax, :gaussian

(:linear, :logistic, :poisson, :L1, :quantile, :softmax, :gaussian)

In [445]:
NeuralNetworkClassifier()

NeuralNetworkClassifier(
    builder = Short(
            n_hidden = 0,
            dropout = 0.5,
            σ = NNlib.σ),
    finaliser = NNlib.softmax,
    optimiser = Flux.Optimise.ADAM(0.001, (0.9, 0.999), IdDict{Any, Any}()),
    loss = Flux.Losses.crossentropy,
    epochs = 10,
    batch_size = 1,
    lambda = 0.0,
    alpha = 0.0,
    rng = Random._GLOBAL_RNG(),
    optimiser_changes_trigger_retraining = false,
    acceleration = CPU1{Nothing}(nothing))

In [455]:
nnmm=machine(
    NeuralNetworkClassifier(builder=MLJFlux.Short(n_hidden=2)),
    x[train,:],y[train])

Machine{NeuralNetworkClassifier{Short,…},…} trained 0 times; caches data
  args: 
    1:	Source @093 ⏎ `Table{AbstractVector{ScientificTypesBase.Continuous}}`
    2:	Source @635 ⏎ `AbstractVector{OrderedFactor{2}}`


In [456]:
# self_tune = TunedModel(
# #     model=LogisticClassifier(penalty=:l1),
# #     model=SVC(),
# #     model=EvoTreeClassifier(),
#     model = NeuralNetworkClassifier(),
    
#     resampling=StratifiedCV(nfolds=5, rng=123),

#     tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
#     range = r,
#     n=2,
#     operation=predict_mode, # use for probabalistic 
#     measure = fnr,  # use for deterministic or with predict_mode for probab 
# #     measure = brier_loss # use for probabalistic 
#     acceleration=CPUThreads(),
# )

# machbest = machine(
#     self_tune,
#     x[train,:],y[train])

# evaluate!(machbest,verbosity=2,resampling=StratifiedCV(nfolds=5, rng=123) )

fit!(nnmm,verbosity=1 )



┌ Info: Training Machine{NeuralNetworkClassifier{Short,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
Optimising neural net:100%[=========================] Time: 0:00:03


Machine{NeuralNetworkClassifier{Short,…},…} trained 1 time; caches data
  args: 
    1:	Source @093 ⏎ `Table{AbstractVector{ScientificTypesBase.Continuous}}`
    2:	Source @635 ⏎ `AbstractVector{OrderedFactor{2}}`


In [462]:
yhat=MLJ.predict_mode(nnmm, x[test,:]); # probab
# yhat=MLJ.predict(machbest, x[test,:]); # determ

In [463]:
ConfusionMatrix()(yhat,y[test])

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │    false    │    true     │
├─────────────┼─────────────┼─────────────┤
│    false    │    1582     │      3      │
├─────────────┼─────────────┼─────────────┤
│    true     │     12      │     30      │
└─────────────┴─────────────┴─────────────┘


In [459]:
# density(x.gt_AF_Mutect[train],alpha=1,bins=30)
# density!(x.gt_AF_Mutect[test],alpha=1,bins=30)

In [460]:
# report(machbest).best_report.feature_importances # Evo Tree
# fitted_params(machbest).best_fitted_params.coefs[sortperm([i[2] for i in fitted_params(machbest).best_fitted_params.coefs])]



In [461]:
aa = DataFrame(x1=yhat,x2=y[test],sample_key=sample_key[test], vaf=x[test,"gt_AF_Mutect"])
# filter!(x-> x.x1==false && x.x2==true ,aa)
aafn=filter(x-> x.x1==false && x.x2==true ,aa)
# # aatp=filter(x-> x.x1==true && x.x2==true ,aa)

# CSV.write("bmtreals1.csv",leftjoin(CSV.read("bmtreals.csv",DataFrame),
#         DataFrame(sample_key=aafn.sample_key, mutectFN = [1 for i in nrow(aafn)])
#         ,on=:sample_key))
# CSV.write("bmtreals1.csv",leftjoin(CSV.read("bmtreals1.csv",DataFrame),
#         DataFrame(sample_key=aatp.sample_key, mutectTP = [1 for i in nrow(aatp)])
#         ,on=:sample_key))


,x1,x2,sample_key,vaf
,Cat…,Cat…,String,Float32
1,false,true,887041_1_100 17:7579472 G>C,0.0388177
2,false,true,761809_1_10 2:25464501 GT>G,0.0777755
3,false,true,887041_T 2:198265526 A>G,27.4522


# Bayes

In [373]:
Turing.setadbackend(:forwarddiff)

:forwarddiff

In [374]:
xb = copy(x);
yb = [i==true for i in copy(y)];

In [375]:
# histogram(xb.TLOD_Mutect,bins=200,yscale=:log10)

In [376]:
# ? Gamma()

In [377]:
# lazyarray(f, x) = LazyArray(Base.broadcasted(f, x))
@model function logreg(X,  y; predictors=size(X, 2))
    #priors
    α ~ Normal(0, 2.5)
    β ~ filldist(TDist(3), predictors)

    #likelihood
#     for i in 1:predictors
#         y[i] ~ BernoulliLogit(α + LinearAlgebra.dot(X[i,:], β) ) # arraydist( LazyArray(@~ BernoulliLogit.(α .+ X * β) ) )
#     end
    y ~ arraydist(LazyArray(Base.broadcasted(x -> BernoulliLogit(x), α .+ X * β) ))
end;


In [378]:
d = train; #vcat(train[findall(x->x,yb[train])],train[1:1425])
cols = ["gt_AF_Mutect","MPOS_Mutect","TLOD_Mutect", "alt_len","MBQ_Mutect_2","pon_pvalue_Mutect" ]

6-element Vector{String}:
 "gt_AF_Mutect"
 "MPOS_Mutect"
 "TLOD_Mutect"
 "alt_len"
 "MBQ_Mutect_2"
 "pon_pvalue_Mutect"

In [379]:
sum(.~ yb[d])

159

In [380]:
sum(yb[d])

32

In [381]:
# findall(x->x,yb[train])

In [382]:
# train[1:385]

In [385]:

# gmm_sampler = Gibbs(PG(100,:p5,:p7,:p10,:p11,:p12), HMC(0.05, 100, :intercept, :p1,:p2,:p3,:p4,:p6,:p8,:p9))

# Sample using HMC.
m = logreg(Matrix(xb[d,cols]), convert.(Float32,yb[d]) )

chain = sample(m, NUTS(), MCMCThreads(), 3000, 5, progress=true)
# chain = sample(m, gmm_sampler, MCMCThreads(), 100, 5, progress=true)
# chain = sample(m, NUTS(50,0.65), MCMCThreads(), 100, 2, progress=true)
# describe(chain)

┌ Info: Found initial step size
│   ϵ = 0.2
└ @ Turing.Inference /Users/alexpanchot/.julia/packages/Turing/uMQmD/src/inference/hmc.jl:188
┌ Info: Found initial step size
│   ϵ = 0.4
└ @ Turing.Inference /Users/alexpanchot/.julia/packages/Turing/uMQmD/src/inference/hmc.jl:188
┌ Info: Found initial step size
│   ϵ = 0.2
└ @ Turing.Inference /Users/alexpanchot/.julia/packages/Turing/uMQmD/src/inference/hmc.jl:188
┌ Info: Found initial step size
│   ϵ = 0.4
└ @ Turing.Inference /Users/alexpanchot/.julia/packages/Turing/uMQmD/src/inference/hmc.jl:188
┌ Info: Found initial step size
│   ϵ = 0.2
└ @ Turing.Inference /Users/alexpanchot/.julia/packages/Turing/uMQmD/src/inference/hmc.jl:188
Sampling (5 threads): 100%|█████████████████████████████| Time: 0:00:00


Chains MCMC chain (3000×19×5 Array{Float64, 3}):

Iterations        = 1001:1:4000
Number of chains  = 5
Samples per chain = 3000
Wall duration     = 6.79 seconds
Compute duration  = 32.89 seconds
parameters        = α, β[1], β[2], β[3], β[4], β[5], β[6]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat    ⋯
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64    ⋯

           α   -4.7911    0.8922     0.0073    0.0139   3192.0418    1.0006    ⋯
        β[1]    4.2014    1.5484     0.0126    0.0249   3294.9792    1.0009    ⋯
        β[2]    0.8790    0.4477     0.0037    0.0056   6067.1676    0.9999    ⋯
        β[3]   -2.7430    1.5691     0.0128    0.0239   3592.7890    1.0005    ⋯
        β[4]   -1.0688    1.1273   

In [386]:
plot(chain)

In [387]:
mean(chain["β[4]"])

-1.0687510941464633

In [388]:
mode(round.(chain["β[4]"],digits=3))


-0.646

In [389]:
# for beta_j in 1:size(xb,2)
#             if abs(mean(chain["β[$(beta_j)]"])) < 0.1
#                 println(beta_j)
#             end
# end

In [390]:
@time abs(mean(chain["β[1]"]))

  0.000095 seconds (5 allocations: 117.359 KiB)


4.201363433625677

In [391]:
function prediction_turing(x, chain)
    # Pull the means from each parameter's sampled values in the chain.


    # Retrieve the number of rows.
    n = size(x,1)

    # Generate a vector to store our predictions.
    v = fill(mean(chain["α"]), n)

    # Calculate the logistic function for each element in the test set.
    

    for beta_j in 1:size(x,2)
        mm = mean(chain["β[$(beta_j)]"])
#         mm = mode(round.(chain["β[$(beta_j)]"],digits=4))
        if abs(mm) > 0.00001
            for i in 1:n
                v[i] += mm * x[i,beta_j] 
            end
        end
    end
        

    v = logistic.(v)
    return v
end;

In [392]:
@time ypred = prediction_turing(xb[test,cols],chain);

  0.048423 seconds (193.60 k allocations: 5.884 MiB, 93.12% compilation time)


In [399]:
yhat2 = [i>.01 ? true : false for i in ypred];

In [400]:
MLJ.ConfusionMatrix()(yhat2,yb[test])

┌ Warning: The classes are un-ordered,
│ using: negative='false' and positive='true'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/measures/confusion_matrix.jl:112


              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │    false    │    true     │
├─────────────┼─────────────┼─────────────┤
│    false    │    1078     │     14      │
├─────────────┼─────────────┼─────────────┤
│    true     │    1951     │     50      │
└─────────────┴─────────────┴─────────────┘


In [494]:
# [print(i," ") for i in train]

In [208]:
asd=yhat2 .> yb[test];

In [209]:
sum(yb[test])

47

In [210]:
yb[test[asd]];

In [211]:
sample_key[test][asd]

89-element Vector{String}:
 "272341_1_10 20:31022552 A>AGGGT"
 "218281_1_5000 20:31022544 C>T"
 "887041_1_5000 4:106197574 GTC>G"
 "218281_1_1000 20:31023292 T>A"
 "736399_1_10 2:198267519 A>C"
 "761809_1_1000 20:31022441 A>AG"
 "272341_1_1000 22:29085168 C>G"
 "809382_1_1000 20:31022544 C>T"
 "736399_1_100 20:31023295 G>C"
 "272341_T 4:106156339 CAGGTTCCTCAGCTT>C"
 "218281_1_100 20:31022544 C>T"
 "272341_1_1000 17:58678016 GA>CC"
 "272341_1_100 22:29085168 C>G"
 ⋮
 "218281_1_1000 20:31023296 A>AG"
 "887041_1_1000 4:106197574 GTC>G"
 "218281_1_1000 20:31022441 A>AGG"
 "272341_T 22:29085168 C>G"
 "809382_1_10 20:31023295 G>C"
 "272341_1_10 17:58678016 GA>CC"
 "809382_1_10 20:31023296 A>AG"
 "736399_1_10 20:31022553 G>T"
 "272341_1_100 17:58678016 GA>CC"
 "761809_1_100 20:31022441 A>AG"
 "736399_1_10 20:31022556 C>T"
 "218281_1_5000 20:31023292 T>A"

In [396]:
yb[train]

102-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [398]:
sample_key[train][yb[train]]

34-element Vector{String}:
 "887041_1_100 20:31017746 CCAG>C"
 "D-1002 X:39923810 G>T"
 "218281_1_10 2:25457243 G>A"
 "761809_1_100 22:29085168 C>G"
 "218281_1_10 4:106193940 A>T"
 "887041_1_5000 20:30946591 CAGA>C"
 "272341_1_1000 20:31024787 T>C"
 "736399_1_5000 20:31022441 AG>A"
 "D-942 19:33792748 G>A"
 "736399_1_100 20:31022441 AG>A"
 "D-1002 X:39922132 A>C"
 "736399_1_10 4:106190822 C>A"
 "887041_1_1000 20:31023301 A>G"
 ⋮
 "D-1002 X:39909190 C>A"
 "D-1002 X:39922009 G>A"
 "736399_1_100 4:106196829 T>G"
 "809382_1_1000 20:31017746 CCAG>C"
 "887041_1_5000 17:74732995 C>G"
 "887041_1_100 4:106197574 GTC>G"
 "809382_1_5000 17:58678009 T>C"
 "D-1002 X:39916503 G>T"
 "887041_1_5000 2:198265526 A>G"
 "D-942 4:106157663 C>T"
 "218281_1_100 4:106180823 C>T"
 "809382_1_100 2:25464444 A>G"